In [1]:
print("test check done!!!")

test check done!!!


In [2]:
import sys, os

# Get project root — one level up from 'research_and_analyst'
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to path:", project_root)

Project root added to path: c:\Users\bhagy\Development\automated-research-generator


In [3]:
from research_and_analysis.utils.model_loader import ModelLoader

In [4]:
model_loader = ModelLoader()

{"timestamp": "2026-01-28T16:35:55.259300Z", "level": "info", "event": "Initializing ApiKeyManager"}
{"timestamp": "2026-01-28T16:35:55.260361Z", "level": "info", "event": "OPENAI_API_KEY loaded successfully from environment"}
{"timestamp": "2026-01-28T16:35:55.260361Z", "level": "info", "event": "GOOGLE_API_KEY loaded successfully from environment"}
{"timestamp": "2026-01-28T16:35:55.260361Z", "level": "info", "event": "GROQ_API_KEY loaded successfully from environment"}
{"path": "C:\\Users\\bhagy\\Development\\automated-research-generator\\research_and_analysis\\config\\configuration.yaml", "keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-28T16:35:55.265818Z", "level": "info", "event": "Configuration loaded successfully"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-28T16:35:55.267938Z", "level": "info", "event": "YAML configuration loaded successfully"}


In [5]:
llm=model_loader.load_llm()

{"provider": "openai", "model": "gpt-4o", "timestamp": "2026-01-28T16:36:01.654962Z", "level": "info", "event": "Loading LLM"}
{"provider": "openai", "model": "gpt-4o", "timestamp": "2026-01-28T16:36:05.599198Z", "level": "info", "event": "LLM loaded successfully"}


In [6]:
llm.invoke("Hi").content

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Hello! How can I assist you today?'

In [7]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

In [20]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

In [21]:
from IPython.display import Image, display

In [14]:
class Analyst(BaseModel):
    name: str = Field(description="Name of the analyst")
    role: str = Field(description="Role of the analyst")
    affiliation: str = Field(description="Primary affiliation of the analyst")
    description:str = Field(description="Area of expertise and background of the analyst")
    
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nExpertise: {self.description}\n"

In [18]:
class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(description="Comprehensive list of analysts with their roles and affiliations")

In [ ]:
class GenerateAnalystState(TypedDict):
    topic: str #research topic
    max_analysts: int #maximum number of analysts to create
    human_analyst_feedback: str #feedback from human analyst
    analysts: List[Analyst] #Analyst asking questions

In [29]:
analyst_instructions = """
You are tasked with creating a set of AI analyst personas. Follow the instructions carefully:

1. First, review the research topic provided:
{topic}

2. Examine any editorial feedback that has been optionally provided tp guide creation of the analysts:
{human_analyst_feedback}

3. Determine the most interesting themes based upon documents and/or feedback above.

4. Pick the top {max_analysts} themes.

5. Assign one analyst persona to each theme, ensuring diversity in roles, affiliations, and expertise.
"""

In [30]:
print(analyst_instructions.format(
        topic="AI in DNA sequencing",
        max_analysts=5,
        human_analyst_feedback="Focus on possibilities and implications of AI integration in genetic research."
    ))


You are tasked with creating a set of AI analyst personas. Follow the instructions carefully:

1. First, review the research topic provided:
AI in DNA sequencing

2. Examine any editorial feedback that has been optionally provided tp guide creation of the analysts:
Focus on possibilities and implications of AI integration in genetic research.

3. Determine the most interesting themes based upon documents and/or feedback above.

4. Pick the top 5 themes.

5. Assign one analyst persona to each theme, ensuring diversity in roles, affiliations, and expertise.



In [27]:
def create_analyst(state: GenerateAnalystState):
    """
    Creates analysts based on the research topic and requirements.
    Args:
        state (GenerateAnalystState): The current state containing topic, max_analysts, and existing analysts.
    """
    topic = state["topic"]
    max_analysts = state["max_analysts"]
    human_analyst_feedback = state["human_analyst_feedback"]
    
    structured_llm = llm.with_structured_output(Perspectives)
    
    system_messages = analyst_instructions.format(
        topic=topic,
        max_analysts=max_analysts,
        human_analyst_feedback=human_analyst_feedback
    )
    
    analysts = structured_llm.invoke([SystemMessage(content=system_messages)] + [
        HumanMessage(content=f"Generate the set of analysts.")])
    
    #writing the list of analysts to state
    return {"analysts": analysts.analysts}
    
    

In [31]:
create_analyst(
    {'topic': 'AI in DNA sequencing',
    'max_analysts': 3,
    'human_analyst_feedback': 'Focus on possibilities and implications of AI integration in genetic research.'
    }
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'analysts': [Analyst(name='Dr. Emily Chen', role='Geneticist', affiliation='National Institute of Health', description='Dr. Emily Chen is a leading geneticist with a focus on the integration of AI in genetic research. Her work primarily explores how AI can enhance the accuracy and speed of DNA sequencing, potentially revolutionizing personalized medicine.'),
  Analyst(name='Dr. Raj Patel', role='AI Research Scientist', affiliation='Google DeepMind', description='Dr. Raj Patel is an AI research scientist specializing in machine learning applications in healthcare. His expertise lies in developing algorithms that can process large genomic datasets, aiming to uncover new insights into genetic diseases and their treatments.'),
  Analyst(name='Dr. Sarah Thompson', role='Bioethicist', affiliation='Harvard University', description='Dr. Sarah Thompson is a bioethicist who examines the ethical implications of AI in genetic research. Her work addresses concerns about privacy, data security, and

In [9]:
def human_feedback():
    pass

In [10]:
def should_continue():
    pass

In [11]:
builder = StateGraph()

NameError: name 'StateGraph' is not defined

In [ ]:
builder.add_node()
builder.add_node()

In [ ]:
builder.edge(START, create_analyst)
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback",
                              should_continue,
                              ["create_analyst",
                               END])

In [ ]:
memory = MemorySaver()

In [ ]:
builder.compile(interrupt_before=["human_feedback"],
                checkpoint = memory)